In [1]:
import os
import time
import argparse
import module3
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torch.nn.init as init

In [2]:
gene_num = 1
tissue_num = 1
k_num = 0

gene_data_name = 'UTMOST_over333_Liver'
gene_list = os.listdir('../../%s/'%gene_data_name)
gene_name = gene_list[gene_num-1]

print(gene_name)

ENSG00000105701.11


In [5]:
def load_data(gene_data_name, gene_name, proc=True):
    np.random.seed(37)
    
    snp_name = '../../%s/%s/%s__snps.npy'%(gene_data_name, gene_name, gene_name)
    gx_name = '../../%s/%s/%s__gene_expression_levels.npy'%(gene_data_name, gene_name, gene_name)
    
    a = np.float32(np.load(snp_name).transpose())
    b = np.float32(np.load(gx_name).transpose())[:,:44]
        
    snp_data = a
    gx_data = np.nan_to_num(b)
    mask_data = np.float32(np.nan_to_num(np.abs(b)+1)>0)
    
    ex_count = np.sum(mask_data, axis=0)
    
    if proc:
        tiss_mean = np.mean(gx_data, axis=0)
        gx_data = (gx_data - tiss_mean)/ex_count
        s = np.arange(len(gx_data))
        np.random.shuffle(s)
        
        snp_data = snp_data[s]
        gx_data = gx_data[s]
        mask_data = mask_data[s]
        
    return snp_data, gx_data, mask_data

In [6]:
snp, gx, mask = load_data(gene_data_name, gene_name, proc=True)
print(np.shape(snp), np.shape(gx), np.shape(mask))
print(gx*mask)

(620, 20995) (620, 44) (620, 44)
[[-0.00972451 -0.          0.04489472 ... -0.         -0.
  -0.        ]
 [ 0.00889056 -0.          0.29502004 ... -0.         -0.
  -0.        ]
 [ 0.01280094 -0.         -0.         ... -0.         -0.
  -0.        ]
 ...
 [ 0.13013291  0.18625815  0.5156967  ... -0.         -0.
  -0.        ]
 [ 0.05276022 -0.          0.03855387 ... -0.         -0.
  -0.        ]
 [ 0.07066246  0.18869776  0.59153634 ... -0.         -0.
  -0.        ]]
